In [64]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [65]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt -O ~/input.txt

--2024-10-04 12:08:24--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt


Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘/Users/josemiguelvilchesfierro/input.txt’

/Users/josemiguelvi 100%[===================>]   1.06M  --.-KB/s    in 0.1s    

2024-10-04 12:08:25 (10.8 MB/s) - ‘/Users/josemiguelvilchesfierro/input.txt’ saved [1115394/1115394]



In [66]:
# read it in to inspect it
with open('/Users/josemiguelvilchesfierro/input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [132]:
#Hyperparameters
batch_size = 32 # number of sequences in a mini-batch
block_size = 8  # number of characters in a sequence
max_iter = 3000 # number of training iterations
eval_interval = 300
learning_rate = 1e-2
eval_iters = 200
n_emb = 32
head_size = 16

In [68]:
list_ch = sorted(list(set(text)))
vocab_size = len(list_ch)
stoi = {ch:i for i,ch in enumerate(list_ch)}
itos = {i:ch for i,ch in enumerate(list_ch)}
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda h: ''.join([itos[i] for i in h])

In [69]:
#Encode dataset
text_encode = torch.tensor(encode(text), dtype=torch.long)
#Create a dataset
n = int(len(text_encode) * 0.9)
train = text_encode[:n]
valid = text_encode[n:]

In [70]:
torch.manual_seed(1337)
def get_batch(split):
    data = train if split == 'train' else valid
    ix = torch.randint(len(data)- block_size, (batch_size,))
    X = torch.stack([data[   i : i+block_size   ] for i in ix])
    Y = torch.stack([data[i + 1: i+block_size +1] for i in ix])
    return X, Y

In [71]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [137]:
class Head(nn.Module):
    #One head of the transformer

    def __init__(self, head_size):
        super().__init__()
        self.query = nn.Linear(n_emb, head_size, bias=False)
        self.key = nn.Linear(n_emb, head_size, bias=False)
        self.value = nn.Linear(n_emb, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
    
    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out
    
#---------------------------------------------------------------------------
   
class multihead(nn.Module):
    #Multihead transformer

    def __init__(self, head_size, n_heads):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(n_heads)])
    
    def forward(self, x):
        h = torch.cat([h(x) for h in self.heads], dim=-1)
        return self.ln(x + h)
#---------------------------------------------------------------------------

class feedforward(nn.Module):
    #Feedforward layer

    def __init__(self, n_emb):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_emb, 4*n_emb),
            nn.ReLU(),
            nn.Linear(4*n_emb, n_emb)
        )

    def forward(self, x):
        return self.net(x)
#---------------------------------------------------------------------------

class block(nn.Module):
    #Transformer block
    def __init__(self, n_emb, n_heads):
        super().__init__()
        head_size = n_emb // n_heads
        self.mha = multihead(head_size, n_heads)
        self.ff = feedforward(n_emb)

    def forward(self, x):
        x = x + self.mha(x)
        x = x + self.ff(x)
        return x

#---------------------------------------------------------------------------

class BigramLanguajeModel(nn.Module):

    def __init__(self):
        super().__init__()
        self.table_embedding_token = nn.Embedding(vocab_size, n_emb)
        self.table_embedding_position = nn.Embedding(block_size, n_emb)
        self.head = block(head_size, 4)
        self.lm_head = nn.Linear(n_emb, vocab_size) 

    def forward(self, idx, targets= None):
        B, T = idx.shape  
        #Embedding tokens and targets in a 3d tensor
        emb = self.table_embedding_token(idx) # B T C [4x 8 x 32]
        pos = self.table_embedding_position(torch.arange(T, device=idx.device))  # [T, n_emb]
        X = emb + pos
        #Linear layer
        X = self.head(X)
        logits = self.lm_head(X) # B T C [4 x 8 x 65]

        
        if targets is None:
            loss = None 
        else:
            B , T, C = logits.shape
            #Reshape the tensor to 2d for cross entropy
            logits = logits.view(B*T , C) #[32 x 65]
            targets = targets.view(B*T) #[32]
            #Calculate loss
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:] # B x T
            #get the prediction
            logits, _ = self(idx_cond) # [B x T x C] because targets is None [4 x 8 x 65]
            #get the last token
            logits = logits[:, -1, :] # Becomes [B x C] [4 x 65]
            #softmax
            probs = F.softmax(logits, dim=1) 
            #sample
            new_token = torch.multinomial(probs, 1)
            #append to the sequence
            idx = torch.cat([idx, new_token], dim=1) # B x T+1   

        return idx

In [138]:
model = BigramLanguajeModel()

In [114]:
optimier = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [139]:
for i in range(max_iter):

    # every once in a while evaluate the loss on train and val sets
    if (i % eval_interval == 0) or (i == max_iter - 1):
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    #forward step
    xb, yb = get_batch('train')
    logits, loss = model.forward(xb, yb)


    # Print the loss every 1000 epochs
    #if i % 1000 == 0:
    #    print(f'Epoch [{i}/{max_iter}], Loss: {loss.item():.4f}')
    
    #backward step
    optimier.zero_grad(set_to_none=True)
    loss.backward()
    optimier.step()


step <built-in function iter>: train loss 4.1795, val loss 4.1806
step <built-in function iter>: train loss 4.1784, val loss 4.1765
step <built-in function iter>: train loss 4.1772, val loss 4.1782
step <built-in function iter>: train loss 4.1791, val loss 4.1762
step <built-in function iter>: train loss 4.1774, val loss 4.1785
step <built-in function iter>: train loss 4.1792, val loss 4.1772
step <built-in function iter>: train loss 4.1788, val loss 4.1795
step <built-in function iter>: train loss 4.1773, val loss 4.1799
step <built-in function iter>: train loss 4.1777, val loss 4.1775
step <built-in function iter>: train loss 4.1772, val loss 4.1815
step <built-in function iter>: train loss 4.1765, val loss 4.1788


In [140]:
model = BigramLanguajeModel()
idx = torch.zeros((1,1), dtype=torch.long)
generate = model.generate(idx, 400)[0].tolist()
print(decode(generate))



L'&-mxPDK3naM3Pyu,E,hCMRvdGTz;-BHn3&jP
IVqv
fvM:'xwxae'WSsi
AOOKHfy
.P!MA;jzqbFZIMIykQqkL.aWhcgX'Su$eOPNUNe:vH-VNhxqxGkCg;pE-'b!!,mRvb&GD&'-e
&.lpaOGXNmena-qEvHl FGYkXiuRCxsZJzcr-HKEsLG&DF&apnQ;G-,kGzdg'dHnGDRbllh.T!o;ZRo,YcpLu,QfYxB.XoA&PlbovnhRHDxb,vV-rAs,kkTWylOo,Zo?DI
xLMSqwuB;JearSCKRQBhGRajdLNBttM&y
jpAtuLW?,aBhCK.i!kfB$nKkLrG-MK3ZX'GBoyzjlQ

!AA-mBulex'dRQsQPx!Jp-blLqs?wTAVZPsLqCiZPJoMozUG


In [ ]:
#X, Y = get_batch('train')
#crete a tensor with 0s
torch.manual_seed(1337)
B, T, C = 4, 8, 32
X = torch.randn(B, T, C)

#I have to create query and key tensors
head_size = 16
query = torch.nn.Linear(C, head_size, bias= False)
key = torch.nn.Linear(C, head_size, bias= False)
value= torch.nn.Linear(C, head_size, bias= False)
k = key(X)
q = query(X)
#Calculate the attention
wei = q @ k.transpose(-2, -1) 


tril = torch.tril(torch.ones(T, T))
#wei = torch.zeros((T,T))
wei = torch.masked_fill(wei, tril == 0, float('-inf'))
wei = F.softmax(wei, dim=1)
v = value(X)
out = wei @ v

#out = wei @ X

In [ ]:
out.shape

torch.Size([4, 8, 16])

In [ ]:
wei[0]

tensor([[0.0078, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0124, 0.0053, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0803, 0.0310, 0.2812, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3961, 0.8178, 0.1780, 0.2202, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0157, 0.0031, 0.0974, 0.2941, 0.5470, 0.0000, 0.0000, 0.0000],
        [0.3248, 0.0790, 0.0623, 0.1598, 0.1693, 0.0106, 0.0000, 0.0000],
        [0.1338, 0.0264, 0.2808, 0.1414, 0.1035, 0.5438, 0.2053, 0.0000],
        [0.0290, 0.0374, 0.1003, 0.1846, 0.1802, 0.4456, 0.7947, 1.0000]],
       grad_fn=<SelectBackward0>)

In [ ]:
T = X.shape[1]
position_embedding_table = nn.Embedding(block_size, n_emb)
pos_emb = position_embedding_table(torch.arange(T))

In [ ]:
pos_emb.shape

torch.Size([8, 32])

In [ ]:
(token_emb + pos_emb).shape

torch.Size([32, 8, 32])